<a href="https://colab.research.google.com/github/KuntumallaJayashree/Inventory_management/blob/main/Inventory_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 37.0 MB/s eta 0:00:00


In [33]:
import pandas as pd
import pulp as p
from pulp import *

In [34]:
#plant location
#Manufacturing locations
man_var_cost = pd.read_excel("variable_costs.xlsx",index_col = 0)

In [35]:
man_var_cost.head()

,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
Germany,13,13,13,13,13
Japan,10,10,10,10,10
Brazil,8,8,8,8,8
India,5,5,5,5,5


In [36]:
freight_Cost = pd.read_excel("freight_costs.xlsx",index_col = 0)

In [37]:
Variable_cost = freight_Cost/1000 + man_var_cost

In [38]:
Variable_cost

,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,12.000,24.25,13.100,28.100,20.778
Germany,26.335,13.00,21.617,33.244,23.073
Japan,25.400,32.75,10.000,53.610,24.350
Brazil,24.450,30.05,36.000,8.000,37.750
India,18.650,20.40,29.500,34.400,5.000


In [39]:
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col = 0)
fixed_costs

,Low,High
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


In [40]:
#Planr capacity
cap = pd.read_excel('capacity.xlsx', index_col = 0)
cap

,Low,High
Capacity (kUnits/month),,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,3000


In [41]:
## DEMAND
demand = pd.read_excel('demand.xlsx', index_col = 0)
demand

,Demand
(Units/month),
USA,2800000
Germany,90000
Japan,1700000
Brazil,145000
India,160000


In [42]:
#Defining decision variables:
loc = ['USA','Germany','Japan','Brazil','India']
size = ['Low','High']

In [43]:
#initializing the class of linear programming problem
LP_Prob = p.LpProblem("Capacitated Plant located model",LpMinimize)

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Decision Variables
LpVariable(name, indexs, lowBound=None, upBound=None, cat='Continuous')
LpVariable.dicts() oen used with Python's list comprehension

x = quantity produced at location
_
i
_ and shipped to _j_
y = 1 if the plant at location
_
i
_ of capacity _s_
is open, 0 if closed
s = low or high capacity plant

Objective function
Minimize z = (f y ) + (c x )
c = cost of producing and shipping from plant _
i
_ to region
_j_
f = xed cost of keeping plant _
i
_ of capacity _s_ open
n = number of production facilities
m = number of markets or regional demand points

In [44]:
# Create Decision Variables
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts("plant_",
                     [(i,s) for s in size for i in loc], cat='Binary')

In [45]:
y

{('USA', 'Low'): plant__('USA',_'Low'),
 ('Germany', 'Low'): plant__('Germany',_'Low'),
 ('Japan', 'Low'): plant__('Japan',_'Low'),
 ('Brazil', 'Low'): plant__('Brazil',_'Low'),
 ('India', 'Low'): plant__('India',_'Low'),
 ('USA', 'High'): plant__('USA',_'High'),
 ('Germany', 'High'): plant__('Germany',_'High'),
 ('Japan', 'High'): plant__('Japan',_'High'),
 ('Brazil', 'High'): plant__('Brazil',_'High'),
 ('India', 'High'): plant__('India',_'High')}

In [46]:
#Define Objective Function

LP_Prob += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] * 1000 for s in size for i in loc])
            + lpSum([Variable_cost.loc[i,j] * x[(i,j)]   for i in loc for j in loc]))

In [47]:
#Adding constraints:::
for j in loc:
  LP_Prob += lpSum([x[(i,j)] for i in loc]) ==demand.loc(j,"Demand") --->>> Do capacity of plant meets demand??
for i in loc:
  LP_Prob += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] * 1000
                                                       for s in size])



In [48]:
# solve model

LP_Prob.solve()

1

In [49]:
print("Total Costs = {:,} ($/Month)".format(int(value(LP_Prob.objective))))

Total Costs = 0 ($/Month)


In [50]:
print('\n' + "Status: {}".format(LpStatus[LP_Prob.status]))


Status: Optimal


In [52]:
LP_Prob.variables()

[plant__('Brazil',_'High'),
 plant__('Brazil',_'Low'),
 plant__('Germany',_'High'),
 plant__('Germany',_'Low'),
 plant__('India',_'High'),
 plant__('India',_'Low'),
 plant__('Japan',_'High'),
 plant__('Japan',_'Low'),
 plant__('USA',_'High'),
 plant__('USA',_'Low'),
 production__('Brazil',_'Brazil'),
 production__('Brazil',_'Germany'),
 production__('Brazil',_'India'),
 production__('Brazil',_'Japan'),
 production__('Brazil',_'USA'),
 production__('Germany',_'Brazil'),
 production__('Germany',_'Germany'),
 production__('Germany',_'India'),
 production__('Germany',_'Japan'),
 production__('Germany',_'USA'),
 production__('India',_'Brazil'),
 production__('India',_'Germany'),
 production__('India',_'India'),
 production__('India',_'Japan'),
 production__('India',_'USA'),
 production__('Japan',_'Brazil'),
 production__('Japan',_'Germany'),
 production__('Japan',_'India'),
 production__('Japan',_'Japan'),
 production__('Japan',_'USA'),
 production__('USA',_'Brazil'),
 production__('USA',_'

In [54]:
dict_plant = {}
dict_prod = {}
for v in LP_Prob.variables():
    if 'plant' in v.name:
        name = v.name.replace('plant__', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        p_name = name
    else:
        name = v.name.replace('production__', '').replace('_', '')
        dict_prod[name] = v.varValue
    print(name, "=", v.varValue)


('Brazil','High') = 0.0
('Brazil','Low') = 0.0
('Germany','High') = 0.0
('Germany','Low') = 0.0
('India','High') = 0.0
('India','Low') = 0.0
('Japan','High') = 0.0
('Japan','Low') = 0.0
('USA','High') = 0.0
('USA','Low') = 0.0
('Brazil','Brazil') = 0.0
('Brazil','Germany') = 0.0
('Brazil','India') = 0.0
('Brazil','Japan') = 0.0
('Brazil','USA') = 0.0
('Germany','Brazil') = 0.0
('Germany','Germany') = 0.0
('Germany','India') = 0.0
('Germany','Japan') = 0.0
('Germany','USA') = 0.0
('India','Brazil') = 0.0
('India','Germany') = 0.0
('India','India') = 0.0
('India','Japan') = 0.0
('India','USA') = 0.0
('Japan','Brazil') = 0.0
('Japan','Germany') = 0.0
('Japan','India') = 0.0
('Japan','Japan') = 0.0
('Japan','USA') = 0.0
('USA','Brazil') = 0.0
('USA','Germany') = 0.0
('USA','India') = 0.0
('USA','Japan') = 0.0
('USA','USA') = 0.0


In [55]:
# Capacity Plant
list_low, list_high = [], []
for l in loc:
    for cap in ['Low', 'High']:
        x = "('{}','{}')".format(l, cap)
        if cap == 'Low':
            list_low.append(dict_plant[x])
        else:
            list_high.append(dict_plant[x])
df_capacity = pd.DataFrame({'Location': loc, 'Low': list_low, 'High': list_high}).set_index('Location')

df_capacity

,Low,High
Location,,
USA,0,0
Germany,0,0
Japan,0,0
Brazil,0,0
India,0,0
